In [2]:
import math
import pandas as pd

In [ ]:
data = pd.read_excel('./dataset/Coffee Shop Sales.xlsx')
data.to_csv('./dataset/Coffee Shop Sales.csv', index=False)

In [3]:
data2 = pd.read_csv('./dataset/Coffee Shop Sales.csv')

In [4]:
columns = data2.columns
size_per_column = dict()
for column in columns:
    elements = data2[column].unique()
    elements_size = [len(str(elem)) for elem in elements]
    size_per_column[column] = max(elements_size)
print(size_per_column)
print(list(size_per_column.values()))

{'transaction_id': 6, 'transaction_date': 10, 'transaction_time': 8, 'transaction_qty': 1, 'store_id': 1, 'store_location': 15, 'product_id': 2, 'unit_price': 5, 'product_category': 18, 'product_type': 21, 'product_detail': 28}
[6, 10, 8, 1, 1, 15, 2, 5, 18, 21, 28]


In [86]:
class Fixed_Size_Heap:
    def __init__(self, block_size, field_sizes, filename):
        self.block_size = block_size
        self.field_sizes = field_sizes
        self._set_record_size()
        self.blocks = []
        self._read_file(filename)
        self.deleted_records = []

    def _set_field_names(self, line):
        self.field_names = line.split(',')

    def _set_record_size(self):
        self.record_size = sum(self.field_sizes) + len(self.field_sizes)
    
    def _padding(self, field, field_id):
        diff = self.field_sizes[field_id] - len(field)
        padded_field = field + (' ' * diff)
        return padded_field

    def _format_record(self, record):
        formatted_record = ''
        fields = record.strip().split(',')
        for i in range(len(fields)):
            if len(fields[i]) < self.field_sizes[i]:
                padded_field = self._padding(fields[i], i)
                formatted_record += padded_field + ','
            else:
                formatted_record += fields[i] + ','
        return formatted_record

    def _write_record(self, record):
        if self.blocks == []:
            self.blocks.append(self._format_record(record))
        elif len(self.blocks[-1]) + self.record_size < self.block_size:
            self.blocks[-1] += self._format_record(record)
        else:
            self.blocks.append(self._format_record(record))

    def _read_file(self, filename):
        with open(filename, 'r') as file:
            self._set_field_names(file.readline())
            for record in file:
                self._write_record(record)
    
    def _search(self, field_id, value):
        field_size = self.field_sizes[field_id]
        number_of_records = math.floor(self.block_size / self.record_size)
        success = False
        for i in range(len(self.blocks)):
            if success and field_id == 0:
                break
            for j in range(0, number_of_records):
                offset = self.record_size * j
                if self.field_sizes[:field_id] != []:
                    offset += sum(self.field_sizes[:field_id]) + field_id
                field_value = self.blocks[i][offset:offset + field_size].strip()
                if field_value == '':
                    continue
                if field_value == value:
                    yield [i, j]
                    success = True
                    if field_id == 0:
                        break
        return [-1, -1]

    def _select(self, select_container, block_id, record_id):
        offset = self.record_size * record_id
        record = self.blocks[block_id][offset:offset + self.record_size]
        select_container.append(record)

    def select_by_single_primary_key(self, key):
        select_container = []
        for (i, j) in self._search(field_id=0, value=key):
            if i == -1 and j == -1:
                raise Exception('SelectionError: Primary Key nonexistent.')
            else:
                self._select(select_container=select_container, block_id=i, record_id=j)
        return select_container
    
    def select_by_multiple_primary_key(self, keys):
        select_container = []
        exception_counter = 0
        for key in keys:
            for (i, j) in self._search(field_id=0, value=key):
                if i == -1 and j == -1:
                    exception_counter += 1
                else:
                    self._select(select_container=select_container, block_id=i, record_id=j)
        if exception_counter == len(keys):
            raise Exception('SelectionError: Primary Keys nonexistent.')
        return select_container
    
    def select_by_field_interval(self, field, lower_limit, upper_limit):
        field_id = self.field_names.index(field)
        select_container = []
        exception_counter = 0
        for value in range(int(lower_limit), int(upper_limit)):
            for (i, j) in self._search(field_id=field_id, value=str(value)):
                if i == -1 and j == -1:
                    exception_counter += 1
                else:
                    self._select(select_container=select_container, block_id=i, record_id=j)
        if exception_counter == int(upper_limit) - int(lower_limit):
            raise Exception('SelectionError: Requested Records nonexistent.')
        return select_container

    def _delete_record(self, block_id, record_id):
        offset = self.record_size * record_id
        head = self.blocks[block_id][:offset]
        body = ' ' * self.record_size
        tail = self.blocks[block_id][offset + self.record_size:]
        self.blocks[block_id] = head + body + tail
        self.deleted_records.append([block_id, record_id])

    def delete_record_by_primary_key(self, key):
        for (i, j) in self._search(field_id=0, value=key):
            if i == -1 and j == -1:
                raise Exception('DeleteError: Primary Key nonexistent.')
            else:
                self._delete_record(block_id=i, record_id=j)
    
    def delete_record_by_criterion(self, field, value):
        field_id = self.field_names.index(field)
        for (i, j) in self._search(field_id=field_id, value=value):
            if i == -1 and j == -1:
                raise Exception('DeleteError: Field Value nonexistent.')
            else:
                self._delete_record(block_id=i, record_id=j)

In [37]:
filename = './dataset/Coffee Shop Sales.csv'

In [81]:
myfile = Fixed_Size_Heap(
                    block_size=512,
                    field_sizes=list(size_per_column.values()),
                    filename=filename)

In [49]:
myfile.blocks[0]

'1     ,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0  ,Coffee            ,Gourmet brewed coffee,Ethiopia Rg                 ,2     ,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1  ,Tea               ,Brewed Chai tea      ,Spicy Eye Opener Chai Lg    ,3     ,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5  ,Drinking Chocolate,Hot chocolate        ,Dark chocolate Lg           ,4     ,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0  ,Coffee            ,Drip coffee          ,Our Old Time Diner Blend Sm ,'

In [ ]:
myfile.delete_record_by_primary_key('3')

In [44]:
myfile.blocks[1]

'5     ,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1  ,Tea               ,Brewed Chai tea      ,Spicy Eye Opener Chai Lg    ,6     ,2023-01-01,07:22:41,1,5,Lower Manhattan,77,3.0  ,Bakery            ,Scone                ,Oatmeal Scone               ,                                                                                                                                                                                                                                                            '

In [42]:
myfile.delete_record_by_criterion(field='product_category', value='Coffee')

In [9]:
myfile.delete_record_by_criterion(field='transaction_date', value='2023-01-01')

In [50]:
select_container = myfile.select_by_single_primary_key('3')
print(select_container)

['3     ,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5  ,Drinking Chocolate,Hot chocolate        ,Dark chocolate Lg           ,']


In [55]:
select_container = myfile.select_by_multiple_primary_key(['3', '5', '6'])
print(select_container)

['3     ,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5  ,Drinking Chocolate,Hot chocolate        ,Dark chocolate Lg           ,', '5     ,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1  ,Tea               ,Brewed Chai tea      ,Spicy Eye Opener Chai Lg    ,', '6     ,2023-01-01,07:22:41,1,5,Lower Manhattan,77,3.0  ,Bakery            ,Scone                ,Oatmeal Scone               ,']


In [85]:
select_container = myfile.select_by_field_interval(field='store_id', lower_limit='5', upper_limit='8')
print(select_container[:5])
print(len(select_container))

5
6
7
['1     ,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0  ,Coffee            ,Gourmet brewed coffee,Ethiopia Rg                 ,', '2     ,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1  ,Tea               ,Brewed Chai tea      ,Spicy Eye Opener Chai Lg    ,', '3     ,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5  ,Drinking Chocolate,Hot chocolate        ,Dark chocolate Lg           ,', '4     ,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0  ,Coffee            ,Drip coffee          ,Our Old Time Diner Blend Sm ,', '5     ,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1  ,Tea               ,Brewed Chai tea      ,Spicy Eye Opener Chai Lg    ,']
47782
